# Download Apple Watch Data

Example for how to download accelerometry, rotation, tremor, and dyskinesia from Apple Watch.

In [1]:
import io
import os
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from runeq import Config, stream

In [2]:
def make_full_df(accessor):
    """Loop through pages of API calls and append to a single dataframe"""

    df = pd.DataFrame()
    for page in accessor.iter_csv_text():
        page_df = pd.read_csv(io.StringIO(page))
        df = df.append(page_df, ignore_index=True)

    return df


def get_accel(client, params, save_filepath):
    """Makes API calls for accel data, saves to CSV and outputs dataframe"""

    accessor = client.Accel(**params)
    df = make_full_df(accessor)
    df.to_csv(save_filepath, index=False)

    return df


def get_rotation(client, params, save_filepath):
    """Makes API calls for rotation data, saves to CSV and outputs dataframe"""

    accessor = client.Rotation(**params)
    df = make_full_df(accessor)
    df.to_csv(save_filepath, index=False)

    return df


def get_tremor(client, params, save_filepath):
    """Makes API calls for tremor data, saves to CSV and outputs dataframe"""

    accessor = client.ProbabilitySymptom(
        symptom='tremor',
        severity='*',
        **params
    )
    df = make_full_df(accessor)
    df.to_csv(save_filepath, index=False)

    return df


def get_dyskinesia(client, params, save_filepath):
    """
    Makes API calls for dyskinesia data, saves to CSV and outputs
    dataframe
    """

    accessor = client.ProbabilitySymptom(
        symptom='dyskinesia',
        **params
    )
    df = make_full_df(accessor)
    df.to_csv(save_filepath, index=False)

    return df

In [3]:
# set up client

cfg = Config()
client = stream.V1Client(cfg)

Times of interest:

**RCS02**
* Open Loop:
Left: 5/26/20-5/30/20
* Closed Loop:
Left: 6/10/20-6/15/20

**RCS05**:
* Left: 6/16/20 - 6/20/20
* Right: 6/16/20, 6/18/20, 6/19/20

**RCS06**:
* Open Loop:
Both sides: 6/4/20-6/5/20
* Closed Loop:
Left: 6/15

**RCS07**:
* Both: 6/23/20-6/27/20

**RCS08**:
* Both: 6/23/20-6/27/20


**+ Everything from this week (week of Sept 20) from**: 
RCS02, RCS05, RCS07, RCS08 

In [4]:
# specify patient ids and device ids

subjects = [
    # 'rcs02',
    # 'rcs05',
    # 'rcs06',
    # 'rcs07',
    'rcs08'
]

patient_ids = {
    # 'rcs02': 'RCS02',
    # 'rcs05': 'RCS05',
    # 'rcs06': 'rcs06',
    # 'rcs07': 'rcs07',
    'rcs08': 'ee8ad14f051f4ff0bccab5c98171067f'
}

L_device_ids = {
    # 'rcs02': 'S9Nj2M1Q',
    # 'rcs05': 'fkFVvXvO',
    # 'rcs06': 'CWTFFG3f',
    # 'rcs07': '0Z48CCdv',
    'rcs08': 'WJzJ4*NA'
}

R_device_ids = {
    # 'rcs02': 'KrCk6N6q',
    # 'rcs05': 'yFZe7BQZ',
    # 'rcs06': 'EK5cDAZ1',
    # 'rcs07': 'RElEtNme',
    'rcs08': 'fMQ45ejQ'
}

Here's an example for how you would set up parameters for the L side for all subjects, for the last week.

For the other dates of interest, you can set up another dictionary with patient/side-specific date ranges, and introduce additional loops.

In [5]:
# set L parameters for week of Sept 20

L_params = {}

for subject in subjects:
    L_params[subject] = {
        'patient_id': patient_ids[subject],
        'device_id': L_device_ids[subject],
        'start_time': datetime.datetime(2020, 10, 7).timestamp(),
        'end_time': datetime.datetime(2020, 10, 11).timestamp(),
        'timestamp': 'datetime'
    }

In [6]:
# specify base path for saving data

BASE_PATH = '/Users/roee/Downloads/watch'

In [7]:
# download data

for subject in subjects:
    accel = get_accel(
        client,
        L_params[subject],
        os.path.join(BASE_PATH, 'ex_accel.csv')
        subject + '_accel'
    )
    rotation = get_rotation(
        client,
        L_params[subject],
        os.path.join(BASE_PATH, 'ex_rotation.csv')
    )
    tremor = get_tremor(
        client,
        L_params[subject],
        os.path.join(BASE_PATH, 'ex_tremor.csv')
    )
    dyskinesia = get_dyskinesia(
        client,
        L_params[subject],
        os.path.join(BASE_PATH, 'ex_dyskinesia.csv')
    )

SyntaxError: invalid syntax (<ipython-input-8-a731ee568ce6>, line 2)


The way the functions are written, they currently save single queries into one CSV. For particularly large queries, you may want to save separate pages of the request into separate CSV's (instead of `df.append`, write the page into a file).